# SteganoGAN in Keras
This notebook contains code attempting to reimplement SteganoGAN in Keras, for the purpose of better understanding (and scrutinizing) it.

*Based on https://github.com/DAI-Lab/SteganoGAN/tree/master/steganogan*

### Modules

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import sys

import tensorflow as tf
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError

from models import ResidualEncoder, BasicDecoder, Critic

from resnet_steganogan_gp import ResnetSteganoGAN

2025-01-28 01:52:27.248255: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Constants

In [2]:
# Image dimensions
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
IMAGE_CHANNELS = 3

IMAGE_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)
MESSAGE_DEPTH = 6
BATCH_SIZE = 4
MODEL_PATH = 'ResnetSteganoGAN.weights.h5'

IMAGES_TESTING_PATH = 'images/testing'
IMAGES_OUTPUT_PATH = 'images/testing_output'

----

### Call main encode and decode functions (with creating steganographic image and decoding it)

In [3]:
encoder = ResidualEncoder(MESSAGE_DEPTH)
decoder = BasicDecoder(MESSAGE_DEPTH)
critic = Critic()

encoder.build(input_shape=[(None, None, None, IMAGE_CHANNELS), (None, None, None, MESSAGE_DEPTH)])
decoder.build(input_shape=(None, None, None, IMAGE_CHANNELS))
critic.build(input_shape=(None, None, None, IMAGE_CHANNELS))

resnetSteganoGAN = ResnetSteganoGAN(
  encoder=encoder,
  decoder=decoder,
  critic=critic,
  data_depth=MESSAGE_DEPTH
)

resnetSteganoGAN.build(input_shape=[(None, None, None, IMAGE_CHANNELS), (None, None, None, MESSAGE_DEPTH)])

if MODEL_PATH is not None and os.path.exists(MODEL_PATH):
  resnetSteganoGAN.load_weights(MODEL_PATH)
  print(f'Model loaded from {MODEL_PATH}')

resnetSteganoGAN.compile(
  encoder_optimizer  = Adam(learning_rate=1e-4),
  decoder_optimizer  = Adam(learning_rate=1e-4),
  critic_optimizer   = Adam(learning_rate=1e-4, beta_1=0.5, beta_2=0.9),
  similarity_loss_fn = MeanSquaredError(),
  decoder_loss_fn    = BinaryCrossentropy(from_logits=False) # false means that data to compare is in [0, 1]
)

Model loaded from ResnetSteganoGAN.weights.h5


In [4]:
resnetSteganoGAN.encode(f'{IMAGES_TESTING_PATH}/input1.png', f'{IMAGES_OUTPUT_PATH}/output1.png', 'Hello, World! 1111')
print(resnetSteganoGAN.decode(f'{IMAGES_OUTPUT_PATH}/output1.png'))

Found 1 candidates for message, choosing most common.
°2qIÌýd¾¿'¢zc ³1¶z! {°u2ÓÍgÝcü1×¯p~00²6¸Ixýeÿ¿5{n 1õ|0XW¸y|ýÿ*ã£(p p{ aÞc­=W¯r~ðp÷5/xÓHö½i÷¿ §zk 1õ|0ZGø4aÿ<àWÿ£(p x°ñ4Óeßÿk­2Û¯1~à8òv 8ÓÁHç½eÿ¿5¶zk 1õ¼0xK8¥xÝ¿6âWÿ£¨b ±ðu ÑÍeß¼c©1_¯0~°0uw	A|Öýe¿7¦~c 1u T¼!tÝß6¡ÿ£hb òpv 	eßégý1×¯0þp4q6	 8eÜÒíd½&zb ³a ¼qh¿4¡ÿ#hâ ²pg	 ÑIåùç1/2~
 1C÷= 4±hü?<°WCètqâu p} Ícìïk­5S.3~UftsuQ\ô¯`ïý07ÿrm¤1ó=±pÜ?4ñW#®tAâuCðy a¼®k½%W.2þbtó= TÔ®`ïÿ4¶+rm¤1÷= 4 Å±h¼¿4ð×Cht!â1p}ÁÍeüïk­qW®stâtsuÅVõ¯`ïý0¶«r-ã1÷= pÅñ`üÿ,°×#ht!cuQ°} Íe,ík­eW®s|ât÷5 ÅTõ¯`ÿÿ0¶ër-¡1÷= pÅõ`üÿ,ð×jt!â5p}ÄÍa|ïk­uW®s|ât÷5ÅTõ¯`ÿý4¶ëv-e1÷= pÅõ`Ýï<ð×htaâ5Sp}ÅÍeüík­qW®stâtóuQÅVõ­`ÿÿ4·ëv-å1÷= ðÅõ`üÿ<°WCjt!â5p}ÅÍg|ïk­uW®stwâtó=ÅQÅVõ­pïÿ4÷ër-e5A÷= 0ñ`Üÿ,ð×#jt1â5SpuÅÍe|ïk­uW®stwât÷5ÅVõ¯pÿý0¶ëv-c1÷= ðÅõ`Ýï<ð×Ckt!â5p=Åaìïk­!W®r|ât

In [5]:
resnetSteganoGAN.encode(f'{IMAGES_TESTING_PATH}/input2.png', f'{IMAGES_OUTPUT_PATH}/output2.png', 'Hello, World! 2222')
print(resnetSteganoGAN.decode(f'{IMAGES_OUTPUT_PATH}/output2.png'))

In [ ]:
resnetSteganoGAN.encode(f'{IMAGES_TESTING_PATH}/input3.png', f'{IMAGES_OUTPUT_PATH}/output3.png', 'Hello, World! 3333')
print(resnetSteganoGAN.decode(f'{IMAGES_OUTPUT_PATH}/output3.png'))

In [ ]:
resnetSteganoGAN.encode(f'{IMAGES_TESTING_PATH}/input4.png', f'{IMAGES_OUTPUT_PATH}/output4.png', 'Hello, World! 4444')
print(resnetSteganoGAN.decode(f'{IMAGES_OUTPUT_PATH}/output4.png'))

### SteganoGAN predict random data with metrics

In [ ]:
cover_image = tf.random.uniform([1, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS], -1, 1, dtype=tf.float32)
message = tf.cast(tf.random.uniform([1, IMAGE_HEIGHT, IMAGE_WIDTH, MESSAGE_DEPTH], 0, 2, dtype=tf.int32), tf.float32)

stego_img, recovered_msg = resnetSteganoGAN.predict([cover_image, message])

print("stego_img min: {0}, max: {1}".format(tf.reduce_min(stego_img), tf.reduce_max(stego_img)))
print("recovered_msg min: {0}, max: {1}".format(tf.reduce_min(recovered_msg), tf.reduce_max(recovered_msg)))

print("BinaryCrossentropy: {0}".format(BinaryCrossentropy(from_logits=False)(message, recovered_msg)))
print("PSNR: {0}".format(tf.reduce_mean(tf.image.psnr(cover_image, stego_img, 1))))
print("SSIM: {0}".format(tf.reduce_mean(tf.image.ssim(cover_image, stego_img, 1))))